In [5]:
import json
import numpy as np
import time
from gridfs import GridFS
from pymongo import MongoClient
from PIL import Image
from padma.models.encoder.encoder import EncoderModel, SIZE
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import generate_batch, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
encoder = EncoderModel()

In [8]:
batch_size = 10
limit = 100
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
projection = ['metadata.predictions']
# dict_filtro['metadata.predictions.index'] = False
batch_gen = generate_batch(db, filtro=dict_filtro, projection=projection,
                           batch_size=batch_size, limit=limit)

## Abaixo, faz predição e já grava, registro a registro, no Banco de Dados

In [9]:
X = np.zeros((batch_size, *SIZE, 1), dtype=np.float32)
rows = []
y = []
s = time.time()
for batch, rows in batch_gen:
    if len(batch) == 0:
        break
    for i, (images, row) in enumerate(zip(batch, rows)):
        image_array = encoder.image_prepare(images[0])
        s0 = time.time()
        X[i, :, :, :] = image_array
        rows.append(row)
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    indexes = encoder.model.predict(X)
    indexes = indexes.reshape(-1, 128).astype(np.float32)
    y.append(indexes)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
    ystack = np.vstack(y).astype(np.float32)
    for i in range(batch_size):
        new_list = ystack[i, :].tolist()
        index_row = rows[i]
        _id = row['_id']
        old_predictions = index_row['metadata']['predictions']
        new_predictions = old_predictions
        new_predictions[0]['index'] = new_list
        db.fs.files.update_one({'_id': _id}, {'$set': {'metadata.predicitions': json.dumps(new_predictions)}})
    s3 = time.time()
    print('Atualizou Banco de Dados em %s' % (s3 - s2))
s4 = time.time()
elapsed = s4 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))


Montou X em 0.00 
Fez predição em 0.008016109466552734
Atualizou Banco de Dados em 0.006847858428955078
Montou X em 0.00 
Fez predição em 0.003246307373046875
Atualizou Banco de Dados em 0.0066220760345458984
Montou X em 0.00 
Fez predição em 0.003319263458251953
Atualizou Banco de Dados em 0.007515907287597656
Montou X em 0.00 
Fez predição em 0.003371715545654297
Atualizou Banco de Dados em 0.00655674934387207
Montou X em 0.00 
Fez predição em 0.0033240318298339844
Atualizou Banco de Dados em 0.007073163986206055
Montou X em 0.00 
Fez predição em 0.0032885074615478516
Atualizou Banco de Dados em 0.006376028060913086
Montou X em 0.00 
Fez predição em 0.003262042999267578
Atualizou Banco de Dados em 0.00513458251953125
Montou X em 0.00 
Fez predição em 0.0030984878540039062
Atualizou Banco de Dados em 0.006119966506958008
Montou X em 0.00 
Fez predição em 0.003169536590576172
Atualizou Banco de Dados em 0.006087779998779297
Montou X em 0.00 
Fez predição em 0.0035338401794433594
Atuali